# Session 3 — Prompt Engineering **V2** 중급 전략

이 노트북은 Session 2(V1)에서 학습한 기본 전략 위에 **5가지 V2 중급 전략**을 더해 실습하도록 설계되었습니다.

| 버전 | 전략 | 핵심 개념 |
|---|---|---|
| **V1.1** | **Role Prompting** | 챗봇의 역할을 명확히 부여 |
| **V1.2** | **Sentiment Routing** | 사용자 감정(긍/부정)별 응답 분기 |
| **V1.3** | **Least‑to‑Most Prompting** | 단순→복잡 문제로 단계적 해결 |
| **V1.4** | **Ask for Context** | 부족한 입력을 유도적으로 보완 |
| **V1.5** | **Pre‑warm Chat** | 직전 히스토리 활용 맥락 최적화 |
---
---
> **모델:** `gpt‑4o‑mini` (단일)
>
> **데이터:** 예시 3종(`order_delivery`, `refund`, `account_login`)
>
> **목표:** 각 버전별 **응답·지연 시간·비용**을 비교하고, 최종적으로 V1 + V2 조합을 실험합니다.


## 📦 패키지 설치

In [ ]:
!pip install -q langfuse openai aiohttp nest_asyncio pandas python-dotenv openpyxl nbformat

## ⚙️ 환경 설정 및 Langfuse 초기화

In [6]:
import os, asyncio, time, nest_asyncio, pandas as pd
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
USE_STUB = OPENAI_API_KEY is None

# Langfuse
try:
    from langfuse import Langfuse
    langfuse = Langfuse()
except ModuleNotFoundError:
    langfuse = None
    print("⚠️  langfuse 패키지가 설치되지 않았습니다. Trace 기록이 비활성화됩니다.")

if not USE_STUB:
    from langfuse.openai import AsyncOpenAI
    client = AsyncOpenAI(api_key=OPENAI_API_KEY)
else:
    client = None
    print('🔧  Stub 모드: OPENAI_API_KEY 가 없어 더미 응답 사용')

nest_asyncio.apply()

PRICING = {'input':0.15/1_000_000, 'output':0.60/1_000_000}

async def call_openai(system_p, user_p, chat_history=None, tag='V0'):
    start = time.perf_counter_ns()
    if USE_STUB:
        await asyncio.sleep(0.05)
        answer = f"[STUB {tag}] 응답 예시"
        prompt_tok, completion_tok = 30, 120
    else:
        messages = chat_history[:] if chat_history else []
        messages += [{'role':'system','content':system_p},
                     {'role':'user','content':user_p}]
        resp = await client.chat.completions.create(
            model='gpt-4o-mini',
            messages=messages
        )
        answer = resp.choices[0].message.content.strip()
        usage = resp.usage
        prompt_tok, completion_tok = usage.prompt_tokens, usage.completion_tokens
    latency = (time.perf_counter_ns()-start)/1_000_000
    cost = prompt_tok*PRICING['input'] + completion_tok*PRICING['output']

    return dict(answer=answer, latency_ms=latency,
                prompt_tokens=prompt_tok, completion_tokens=completion_tok,
                usd_cost=cost)


## 📝 예시 시나리오

In [2]:
scenarios = [
    {'scenario':'order_delivery','question':'주문한 상품이 배송 예정일을 지났는데 아직 도착하지 않았어요. 어떻게 확인하나요?'},
    {'scenario':'refund','question':'반품 신청을 했는데 환불 처리가 언제 완료되나요?'},
    {'scenario':'account_login','question':'로그인 시 2단계 인증 오류가 발생합니다. 어떻게 해결할 수 있나요?'}
]
df = pd.DataFrame(scenarios)
df

,scenario,question
0,order_delivery,주문한 상품이 배송 예정일을 지났는데 아직 도착하지 않았어요. 어떻게 확인하나요?
1,refund,반품 신청을 했는데 환불 처리가 언제 완료되나요?
2,account_login,로그인 시 2단계 인증 오류가 발생합니다. 어떻게 해결할 수 있나요?


## 🏃‍♂️ 실행 도우미

In [7]:
async def run(df, version_name, build_sys_prompt, build_user_prompt=lambda r: r['question'],
               chat_history=None):
    tasks=[]
    for _, row in df.iterrows():
        tasks.append(call_openai(build_sys_prompt(row),
                                 build_user_prompt(row),
                                 chat_history=chat_history,
                                 tag=version_name))
    results = await asyncio.gather(*tasks)
    out=df.copy()
    for i,res in enumerate(results):
        for k,v in res.items():
            out.loc[i,f'{version_name}_{k}']=v
    return out

## 🔹 Baseline — V1 (Persona + Tone)

In [16]:
def sys_v1(row):
    return ('You are a calm and professional Korean CS chatbot for an e‑commerce platform. '
            'Answer politely in Korean, max 5 sentences.')
baseline = await run(df, 'V1', sys_v1)
baseline[['scenario','V1_answer']]

,scenario,V1_answer
0,order_delivery,"안녕하세요, 고객님. 상품의 배송 상태를 확인하기 위해서는 주문 확인 페이지에서 배..."
1,refund,안녕하세요. 반품 신청을 해주셔서 감사합니다. 환불 처리는 일반적으로 반품 상품이 ...
2,account_login,"안녕하세요! 2단계 인증 오류가 발생하셨군요. 먼저, 입력하신 인증 코드가 정확한지..."


## 1️⃣ Role Prompting — V1.1

In [17]:
def sys_role(row):
    return ('You are an experienced senior customer‑support agent specialised in '
            f"{row['scenario'].replace('_',' ')} issues. Speak with authority and provide clear next steps in Korean.")
v_role = await run(baseline, 'V1_1', sys_role)
v_role[['scenario','V1_1_answer']]

,scenario,V1_1_answer
0,order_delivery,"고객님, 안녕하세요. 주문하신 상품이 배송 예정일을 지났음에도 불구하고 아직 도착하..."
1,refund,반품 신청 후 환불 처리는 일반적으로 7-14일 이내에 완료됩니다. 반품 제품이 당...
2,account_login,2단계 인증 오류가 발생하는 경우 다음과 같은 단계를 따라 문제를 해결해 보시기 바...


## 2️⃣ Sentiment Routing — V1.2

In [18]:
neg_words={'늦','환불','오류','지연','불만','짜증'}
def detect_sent(q): return 'neg' if any(w in q for w in neg_words) else 'pos'
def sys_sent(row):
    return ('You are a friendly CS chatbot. First apologise sincerely in Korean, '
            'then propose 2 concrete actions.' if detect_sent(row['question'])=='neg'
            else 'You are a cheerful CS chatbot. Answer briefly with 2 bullet points in Korean.')
v_sent = await run(v_role, 'V1_2', sys_sent)
v_sent[['scenario','V1_2_answer']]

,scenario,V1_2_answer
0,order_delivery,- 배송 추적 번호를 확인하여 배송 상태를 확인하세요.\n- 주문한 쇼핑몰 고객센터...
1,refund,죄송합니다. 불편을 드려서 정말 죄송합니다. \n\n환불 처리에 대한 구체적인 안내...
2,account_login,죄송합니다. 귀하의 불편에 대해 진심으로 사과드립니다.\n\n2단계 인증 오류를 해...


## 3️⃣ Least‑to‑Most Prompting — V1.3

In [19]:
def sys_ltm(row):
    return ('Break the problem into simpler hidden steps before answering. '
            'Return only the final concise answer in Korean, numbered 1‑3.')
v_ltm = await run(v_sent, 'V1_3', sys_ltm)
v_ltm[['scenario','V1_3_answer']]

,scenario,V1_3_answer
0,order_delivery,1. 주문한 상품의 배송 추적 번호를 확인합니다.\n2. 해당 배송 업체의 웹사이트...
1,refund,1. 반품 신청 상태 확인: 반품 신청이 승인되었는지 확인합니다. \n2. 반품 배...
2,account_login,"1. 2단계 인증 관련 설정을 확인하고, 올바른 인증 방법이 선택되었는지 확인합니다..."


## 4️⃣ Ask for Context — V1.4

In [20]:
def sys_ask(row):
    return ('If the user question lacks details, politely ask exactly one clarifying question in Korean '
            'before providing a solution; otherwise answer in up to 4 sentences.')
v_ctx = await run(v_ltm, 'V1_4', sys_ask)
v_ctx[['scenario','V1_4_answer']]

,scenario,V1_4_answer
0,order_delivery,어떤 쇼핑몰이나 서비스에서 주문하셨는지 알려주실 수 있나요?
1,refund,"반품 신청 후 환불 처리 기간은 보통 3-7일 정도 소요됩니다. 하지만, 정확한 기..."
2,account_login,어떤 플랫폼에서 2단계 인증 오류가 발생하고 있는지 말씀해 주실 수 있나요?


## 5️⃣ Pre‑warm Chat — V1.5

In [21]:
history=[{'role':'system','content':'Prior conversation: 고객이 배송 지연 문제로 여러 번 문의한 기록이 있음.'}]
def sys_prewarm(row):
    return ('You already know the user felt frustration about delays. Empathise first (1 sentence), '
            'then provide precise tracking steps (≤3 sentences, Korean).')
v_pw = await run(v_ctx, 'V1_5', sys_prewarm, chat_history=history)
v_pw[['scenario','V1_5_answer']]

,scenario,V1_5_answer
0,order_delivery,"배송 지연으로 인해 불편을 드려 정말 죄송합니다. 먼저, 주문 확인 이메일에서 제공..."
1,refund,"불편을 드려 정말 죄송합니다. 반품 신청을 하신 경우, 환불은 물품이 저희 창고에 ..."
2,account_login,"먼저, 2단계 인증 오류로 인해 불편을 겪으신 점 정말 안타깝게 생각합니다. 확인해..."


In [22]:
v_pw

,scenario,question,V1_answer,V1_latency_ms,V1_prompt_tokens,V1_completion_tokens,V1_usd_cost,V1_1_answer,V1_1_latency_ms,V1_1_prompt_tokens,...,V1_4_answer,V1_4_latency_ms,V1_4_prompt_tokens,V1_4_completion_tokens,V1_4_usd_cost,V1_5_answer,V1_5_latency_ms,V1_5_prompt_tokens,V1_5_completion_tokens,V1_5_usd_cost
0,order_delivery,주문한 상품이 배송 예정일을 지났는데 아직 도착하지 않았어요. 어떻게 확인하나요?,"안녕하세요, 고객님. 상품의 배송 상태를 확인하기 위해서는 주문 확인 페이지에서 배...",1755.570250,61.0,92.0,0.000064,"고객님, 안녕하세요. 주문하신 상품이 배송 예정일을 지났음에도 불구하고 아직 도착하...",4037.666416,61.0,...,어떤 쇼핑몰이나 서비스에서 주문하셨는지 알려주실 수 있나요?,781.622458,65.0,20.0,0.000022,"배송 지연으로 인해 불편을 드려 정말 죄송합니다. 먼저, 주문 확인 이메일에서 제공...",1518.089334,87.0,75.0,0.000058
1,refund,반품 신청을 했는데 환불 처리가 언제 완료되나요?,안녕하세요. 반품 신청을 해주셔서 감사합니다. 환불 처리는 일반적으로 반품 상품이 ...,1549.096166,52.0,74.0,0.000052,반품 신청 후 환불 처리는 일반적으로 7-14일 이내에 완료됩니다. 반품 제품이 당...,1976.461750,51.0,...,"반품 신청 후 환불 처리 기간은 보통 3-7일 정도 소요됩니다. 하지만, 정확한 기...",2329.359792,56.0,82.0,0.000058,"불편을 드려 정말 죄송합니다. 반품 신청을 하신 경우, 환불은 물품이 저희 창고에 ...",1601.155792,78.0,71.0,0.000054
2,account_login,로그인 시 2단계 인증 오류가 발생합니다. 어떻게 해결할 수 있나요?,"안녕하세요! 2단계 인증 오류가 발생하셨군요. 먼저, 입력하신 인증 코드가 정확한지...",1700.794416,57.0,92.0,0.000064,2단계 인증 오류가 발생하는 경우 다음과 같은 단계를 따라 문제를 해결해 보시기 바...,4328.061833,57.0,...,어떤 플랫폼에서 2단계 인증 오류가 발생하고 있는지 말씀해 주실 수 있나요?,944.206166,61.0,23.0,0.000023,"먼저, 2단계 인증 오류로 인해 불편을 겪으신 점 정말 안타깝게 생각합니다. 확인해...",2871.494958,83.0,90.0,0.000066


## 📊 Latency & Cost 비교

In [23]:
cols=['scenario']
for v in ['V1','V1_1','V1_2','V1_3','V1_4','V1_5']:
    cols += [f'{v}_latency_ms', f'{v}_usd_cost']
v_pw[cols]

,scenario,V1_latency_ms,V1_usd_cost,V1_1_latency_ms,V1_1_usd_cost,V1_2_latency_ms,V1_2_usd_cost,V1_3_latency_ms,V1_3_usd_cost,V1_4_latency_ms,V1_4_usd_cost,V1_5_latency_ms,V1_5_usd_cost
0,order_delivery,1755.570250,0.000064,4037.666416,0.000174,1303.255875,0.000025,2035.655708,0.000052,781.622458,0.000022,1518.089334,0.000058
1,refund,1549.096166,0.000052,1976.461750,0.000086,2090.640333,0.000091,3858.488708,0.000057,2329.359792,0.000058,1601.155792,0.000054
2,account_login,1700.794416,0.000064,4328.061833,0.000184,3349.277208,0.000128,1997.864000,0.000079,944.206166,0.000023,2871.494958,0.000066


## 🔄 V1 + V2 조합 예시

In [24]:
def sys_combo(row):
    s = sys_role(row)
    s += ' 내부적으로 문제를 3단계로 분해 후 최종 요약만 제공합니다.'
    return s
combo = await run(v_pw,'V2_combo', sys_combo)
combo[['scenario','V2_combo_answer']]

,scenario,V2_combo_answer
0,order_delivery,"고객님, 배송이 지연된 상황에 대해 안내드리겠습니다. 문제를 세 가지 단계로 나누어..."
1,refund,환불 처리는 일반적으로 다음 세 단계로 진행됩니다:\n\n1. **반품 접수 확인*...
2,account_login,로그인 시 2단계 인증 오류를 해결하기 위해 다음 단계를 따르세요:\n\n1. **...


---

## ✍️ 개인 실습 영역

아래 셀을 복제하여 자신만의 **My_V2** 버전을 만들어 보세요.
1. 시나리오 3개 중 1개를 골라 다양한 기법을 조합합니다.  
2. `run()` 함수를 활용해 결과를 추가 열로 기록합니다.


In [ ]:
# TODO: 여기서부터 자유롭게 실험해 보세요
